### Goal :

* Construction d'un fichier json structurant l'ensemble du menu avec :
    * separation en sections (entrées, plats principaux, desserts ...)
    * Identification des plats individuels
    * Extraction des prix
    * Taguer tous les plats selon le regime alimentaire.


In [ ]:
pip install openai

In [ ]:
pip install anthropic

Import

In [ ]:
import os
import requests
import json
import re
import os
from dotenv import load_dotenv
import time
from anthropic import Anthropic
from openai import OpenAI
import json
import re



Load environment variables

In [ ]:
load_dotenv() 

claude_api_key = os.environ.get('claude_api_key')
gpt_api_key = os.environ.get('gpt_api_key')

Paths

In [ ]:
marcello_ocr_result_path = "../data/raw_extracted/Azure_doc_intelligence/result_OCR_marcello.txt"
prima_ocr_result_path = "../data/raw_extracted/Azure_doc_intelligence/result_OCR_prima.txt"
output_dir = '../data/segmented_data'

### Structuration du resultat de l'OCR en utilisant le LLM Claude Haiku

### Functions

In [ ]:
def analyze_menu_openai(ocr_text, prompt,api_key):
    client = OpenAI(api_key=api_key)
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": prompt},
                {"role": "user", "content": ocr_text}
            ],
            temperature=0
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"Erreur: {e}")
        return None

In [ ]:

def analyze_menu_claude(ocr_text, prompt,api_key):
    client = Anthropic(api_key=api_key)
    try:
        response = client.messages.create(
            model="claude-3-5-sonnet-20241022",  # ou "claude-3-7-sonnet-20250219" pour le plus récent
            max_tokens=8192,
            temperature=0,
            system=prompt,  # Le prompt système va ici, pas dans les messages
            messages=[
                {"role": "user", "content": ocr_text}
            ]
        )
        return response.content[0].text
    except Exception as e:
        print(f"Erreur: {e}")
        return None

In [ ]:
def clean_response(response):
    """
    Version minimaliste pour nettoyer les réponses OpenAI.
    
    Args:
        response (dict ou str): Réponse de l'API OpenAI
        
    Returns:
        dict: Menu en JSON
    """
    # Convertir en dict si c'est une string
    if isinstance(response, str):
        response = json.loads(response)
        return response



Definition du prompt

In [ ]:
prompt = """Analyse ce texte OCR de menu et retourne uniquement un JSON valide suivant cette structure:

{{
  "menu": {{
    "sections": [
      {{
        "name": "nom_section",
        "items": [
          {{
            "name": "nom_plat",
            "price": {{"value": 12.50, "currency": null}},
            "description": "description_complète",
            "ingredients": ["ingrédient1", "ingrédient2"],
            "dietary": ["végétarien"]
          }}
        ]
      }}
    ]
  }}
}}

Texte OCR: {ocr_text}

Instructions:
1. Identifie automatiquement les sections (entrées, plats, desserts, pizzas, boissons, etc.)
2. Pour chaque item: nom, prix, description, ingrédients (déduis-les de la description si nécessaire)
3. Prix: utilise uniquement €, $, £, CHF pour currency. Si autre chose ou illisible, mets null

IMPORTANT - Régimes alimentaires (sois très prudent):
- Si tu as un grand doute, laisse dietary vide []
- Règles strictes:
  * "végétarien": AUCUNE viande, poisson, fruits de mer (mais œufs/lait OK)
  * "végétalien": AUCUN produit animal (pas de viande, poisson, œufs, lait, miel, beurre)
  * "sans_gluten": AUCUN blé, orge, seigle, avoine (attention aux sauces, panure)
  * "sans_lactose": AUCUN lait, crème, fromage, beurre, yaourt

ATTENTION - VIANDES (jamais végétarien):
- Jambon, jambon blanc, jambon cru, prosciutto = VIANDE
- Bacon, lardons, pancetta = VIANDE  
- Saucisse, chorizo, pepperoni = VIANDE
- Salami, coppa, bresaola = VIANDE
- Bœuf, porc, agneau, veau = VIANDE
- Poulet, canard, dinde = VIANDE

Exemples:

VÉGÉTARIEN + VÉGÉTALIEN:
- Salade verte simple = ["végétarien", "végétalien"]
- Légumes grillés sans sauce = ["végétarien", "végétalien"] 
- Frites maison = ["végétarien", "végétalien"]
- Soupe de légumes (bouillon végétal) = ["végétarien", "végétalien"]

VÉGÉTARIEN SEULEMENT:
- Pâtes au beurre = ["végétarien"] (beurre = produit laitier)
- Pizza margherita = ["végétarien"] (fromage = produit laitier)
- Omelette = ["végétarien"] (œufs OK pour végétarien)

SANS GLUTEN SEULEMENT:
- Steak grillé nature = ["sans_gluten"] (pas de panure ni sauce)
- Salade de riz = ["sans_gluten"] (riz OK)
- Poisson grillé nature = ["sans_gluten"]

SANS LACTOSE SEULEMENT:
- Pâtes à l'huile d'olive = ["sans_lactose"] (pas de beurre/fromage)
- Viande grillée nature = ["sans_lactose"]

COMBINAISONS:
- Salade de quinoa aux légumes = ["végétarien", "végétalien", "sans_gluten", "sans_lactose"]
- Riz sauté aux légumes = ["végétarien", "végétalien", "sans_gluten", "sans_lactose"]
- Steak frites = ["sans_gluten", "sans_lactose"] (si frites maison)
IMPORTANT : il faut inclure le resultat complet (tous les elements presents dans le texte OCR)
Retourne uniquement le JSON, sans texte additionnel."""

Test on Marcello menu

In [ ]:
with open(marcello_ocr_result_path, "r", encoding="utf-8") as file:
    marcello_ocr_result = file.read()

In [ ]:
# result_marcello = analyze_menu_openai(marcello_ocr_result, prompt,openai_api_key)

In [ ]:
result_marcello = analyze_menu_claude(marcello_ocr_result, prompt,claude_api_key)

In [ ]:
cleaned_result_marcello = clean_response(result_marcello)

In [ ]:
# Enregistrement du resultat dans un fichier
with open(output_dir + "/result_LLM_sonnet_marcello", "w") as file: 
        json.dump(cleaned_result_marcello, file, indent=4, ensure_ascii=False)

Test on Prima lova menu

In [ ]:
with open(prima_ocr_result_path, "r", encoding="utf-8") as file:
    prima_ocr_result = file.read()

In [ ]:
# result_prima = analyze_menu_openai(prima_ocr_result, prompt,openai_api_key)

In [ ]:
result_prima = analyze_menu_claude(prima_ocr_result, prompt,claude_api_key)

In [ ]:
cleaned_result_prima = clean_response(result_prima)

In [ ]:
# Enregistrement du resultat dans un fichier
with open(output_dir + "/result_LLM_sonnet_prima", "w") as file: 
        json.dump(cleaned_result_prima, file, indent=4, ensure_ascii=False)

### Conclusion

* Le modele Haiku de Claude ne peut pas gerer les gros menu a cause de la limite de tokens
* gpt-4o-mini n'est pas limité par la limite de token dans notre cas d'usage mais le temps de traitement dure plus de 2 min pour les gros menu et 22 secondes pour les petit menu (2 fois plus lent que claude haiku)
* resultats legerement superieur pour claude haiku
* Le modele presentant les meilleurs ressultats pour le moment c'est le modele sonnet de Claude (plus rapide, plus performant, peut supporter la limite de toekn pour notre cas d'usage)
* Point d'attention : bien que sonnet presente des bons resultats, les modeles sont trop lent. Trouver une solution.
* Piste a explorer : 
*   Tester le streaming, refaire le prompt pour que le modele nous envoie un format jsonL pour pas avoir de probleme de format de la reponse. 


### Traitement par sections

In [ ]:
# Cell 1: Setup
import os
import json
import re
from anthropic import Anthropic
from dotenv import load_dotenv

load_dotenv()
claude_api_key = os.environ.get('claude_api_key')

client = Anthropic(api_key=claude_api_key)

def call_claude(text: str, prompt: str) -> str:
   response = client.messages.create(
       model="claude-3-5-sonnet-20241022",
       max_tokens=2000,
       temperature=0,
       system=prompt,
       messages=[{"role": "user", "content": text}]
   )
   return response.content[0].text

In [ ]:
# Cell 2: Prompt pour détecter les sections ET le titre
DETECT_SECTIONS_PROMPT = """Analyse ce texte OCR de menu et retourne uniquement un JSON avec les noms des sections et le titre du restaurant/menu.

Format EXACT:
{
 "menu_title": "Nom du Restaurant/Menu",
 "sections": ["SECTION1", "SECTION2", "SECTION3"]
}

Instructions:
1. Identifie le titre/nom du restaurant (généralement en haut du menu)
2. Identifie automatiquement toutes les sections du menu (entrées, plats, desserts, pizzas, boissons, etc.)
3. GARDE EXACTEMENT les noms de sections comme ils apparaissent dans le texte OCR - ne les traduis PAS, ne les modifie PAS
4. Ne retourne QUE le JSON, rien d'autre"""

def detect_sections_and_title(ocr_text: str) -> tuple:
   response = call_claude(ocr_text, DETECT_SECTIONS_PROMPT)
   
   try:
       # Essayer de parser directement
       data = json.loads(response)
       return data["sections"], data["menu_title"]
   except json.JSONDecodeError:
       # Chercher le JSON dans la réponse
       json_match = re.search(r'\{[^}]*"menu_title"[^}]*"sections"[^}]*\}', response)
       if json_match:
           try:
               json_str = json_match.group()
               print(f"JSON extrait: {json_str}")
               data = json.loads(json_str)
               return data["sections"], data["menu_title"]
           except:
               print("❌ Erreur parsing JSON extrait")
       
       # Fallback
       sections = re.findall(r'"([A-Z][A-Z\sÀ-Ü]*)"', response)
       valid_sections = [s for s in sections if len(s.strip()) >= 3]
       return valid_sections, "Menu"

# Test
sections, menu_title = detect_sections_and_title(prima_ocr_result)
print("Titre du menu:", menu_title)
print("Sections détectées:", sections)

In [ ]:
# Cell 3: Étape 2 - Extraire le contenu d'une section (sans le nom) - Version corrigée
import re

def extract_section_content(ocr_text: str, section_name: str, all_sections: list) -> str:
    lines = ocr_text.split('\n')
    content = []
    capturing = False
    
    for line in lines:
        # Vérifier si c'est le début de notre section (mot entier avec frontières)
        if re.search(r'\b' + re.escape(section_name.upper()) + r'\b', line.upper()):
            capturing = True
            # Ne pas ajouter la ligne avec le nom de section
            continue
        elif capturing:
            # Arrêter si on trouve une autre section de la liste détectée (mot entier)
            if any(re.search(r'\b' + re.escape(s.upper()) + r'\b', line.upper()) for s in all_sections if s != section_name):
                break
            content.append(line)
    
    return '\n'.join(content)

# Test - Extraire toutes les sections dans un JSON
sections_with_content = []

for section in sections:
    content = extract_section_content(prima_ocr_result, section, sections)
    sections_with_content.append({
        "name": section,
        "content": content
    })

# Afficher le résultat JSON
result = {"sections": sections_with_content}
print(json.dumps(result, indent=2, ensure_ascii=False))

In [ ]:
# Cell 4: Étape 3 - Analyser chaque section avec temps de traitement
import json
import re
import os
import time

def clean_section_name_for_filename(section_name: str) -> str:
    """Nettoie le nom de section pour créer un nom de fichier valide."""
    # Enlever les caractères spéciaux et espaces
    clean_name = re.sub(r'[^\w\s-]', '', section_name)
    # Remplacer les espaces par des underscores
    clean_name = re.sub(r'\s+', '_', clean_name)
    # Convertir en minuscules
    clean_name = clean_name.lower().strip('_')
    return clean_name

def analyze_section(section_content: str, section_name: str) -> dict:
    # Prompt modifié pour corriger les noms de sections
    ANALYZE_SECTION_PROMPT = f"""Analyse cette section de menu nommée "{section_name}" et retourne uniquement un JSON valide suivant cette structure:

{{
  "name": "nom_section_corrigé",
  "items": [
    {{
      "name": "nom_plat",
      "price": {{"value": 12.50, "currency": "€"}},
      "description": "description_complète",
      "ingredients": ["ingrédient1", "ingrédient2"],
      "dietary": ["végétarien"]
    }}
  ]
}}

Instructions:
1. CORRIGE les erreurs OCR évidentes dans le nom de section "{section_name}":
   - "PRZE" → "PIZZE"
   - "DOLC" → "DOLCI"
   - "ANTPASTI" → "ANTIPASTI"
   - "NSALATE" → "INSALATE"
   - "CARNE" → garde "CARNE" (correct)
   - "PASTA" → garde "PASTA" (correct)
   - etc.
   Utilise le nom corrigé dans le champ "name" du JSON
2. Pour chaque item: nom, prix, description, ingrédients (déduis-les de la description si nécessaire)
3. Prix: utilise uniquement €, $, £, CHF pour currency. Si autre chose ou illisible, mets null
4. DÉTECTION ET TRADUCTION DE LANGUE:
   - Détecte la langue majoritaire du menu
   - Si langue du menu = français → PAS de traduction
   - Si langue du menu = langue avec même alphabet que l'utilisateur → traduis les descriptions MAIS garde les spécialités/ingrédients authentiques en langue originale
   - Si langue du menu = langue avec alphabet différent de l'utilisateur → TRADUIS TOUT car l'utilisateur ne peut pas lire ces caractères
   
    Logique par type d'alphabet:
   
   MÊME FAMILLE D'ALPHABET (garde les spécialités):
   - Latin vers Latin: Italien→Français, Espagnol→Anglais, etc.
   - Cyrillique vers Cyrillique: Russe→Bulgare, etc.
   - Arabe vers Arabe: Arabe→Persan, etc.
   
   ALPHABETS DIFFÉRENTS (traduis tout):
   - Latin vers Chinois: "Carbonara" → "卡邦纳拉意面"
   - Chinois vers Latin: "宫保鸡丁" → "Poulet Gong Bao"
   - Arabe vers Latin: "كباب" → "Kebab"
   - Japonais vers Latin: "寿司" → "Sushi"
   
   Exemples de spécialités à GARDER (même alphabet):
   - Italien→Français: garde "mozzarella di bufala", "parmigiano"
   - Français→Anglais: garde "coq au vin", "bouillabaisse"
   - Espagnol→Italien: garde "jamón ibérico", "paella"

IMPORTANT - Régimes alimentaires (sois très prudent):
- Si tu as un grand doute, laisse dietary vide []
- Règles strictes:
  * "végétarien": AUCUNE viande, poisson, fruits de mer (mais œufs/lait OK)
  * "végétalien": AUCUN produit animal (pas de viande, poisson, œufs, lait, miel, beurre)
  * "sans_gluten": AUCUN blé, orge, seigle, avoine (attention aux sauces, panure)
  * "sans_lactose": AUCUN lait, crème, fromage, beurre, yaourt

ATTENTION - VIANDES (jamais végétarien):
- Jambon, jambon blanc, jambon cru, prosciutto = VIANDE
- Bacon, lardons, pancetta = VIANDE  
- Saucisse, chorizo, pepperoni = VIANDE
- Salami, coppa, bresaola = VIANDE
- Bœuf, porc, agneau, veau = VIANDE
- Poulet, canard, dinde = VIANDE

IMPORTANT: Inclus TOUS les éléments présents dans cette section.
Retourne UNIQUEMENT le JSON, sans texte additionnel."""

    response = call_claude(section_content, ANALYZE_SECTION_PROMPT)
    
    try:
        return json.loads(response)
    except json.JSONDecodeError as e:
        # Fallback avec le bon nom de section
        return {"name": section_name, "items": []}

# Créer le dossier de sortie pour les sections
sections_output_dir = '../data/sections_analyzed'
os.makedirs(sections_output_dir, exist_ok=True)

# Variables pour le suivi du temps total
total_start_time = time.time()
all_sections_data = []

print("🍽️ Début de l'analyse des sections")
print("="*50)

# Traitement complet - boucle sur le JSON des sections
for i, section_data in enumerate(sections_with_content, 1):
    section_name = section_data["name"]
    section_content = section_data["content"]
    
    print(f"[{i}/{len(sections_with_content)}] Traitement: {section_name}")
    
    # Mesurer le temps de traitement de cette section
    section_start_time = time.time()
    
    # Passer le nom de section à la fonction
    analyzed = analyze_section(section_content, section_name)
    
    section_end_time = time.time()
    section_duration = section_end_time - section_start_time
    
    print(f"    ✅ Terminé en {section_duration:.2f}s")
    
    # Récupérer le nom corrigé du JSON retourné
    corrected_section_name = analyzed.get("name", section_name)
    
    # Enregistrer avec le nom corrigé
    clean_filename = clean_section_name_for_filename(corrected_section_name)
    section_filepath = os.path.join(sections_output_dir, f"{clean_filename}.json")
    
    with open(section_filepath, 'w', encoding='utf-8') as file:
        json.dump(analyzed, file, indent=4, ensure_ascii=False)
    
    all_sections_data.append(analyzed)

print("="*50)

# Calcul du temps total
total_end_time = time.time()
total_duration = total_end_time - total_start_time

print(f"⏱️ Temps total: {total_duration:.2f}s")
print(f"📊 Moyenne par section: {total_duration/len(sections_with_content):.2f}s")

# Résultat final
menu = {
    "menu": {
        "name": menu_title,
        "sections": all_sections_data
    }
}

# Enregistrer le menu complet
final_menu_path = os.path.join(sections_output_dir, "menu_complet.json")
with open(final_menu_path, 'w', encoding='utf-8') as file:
    json.dump(menu, file, indent=4, ensure_ascii=False)

print(f"💾 Menu complet sauvegardé: {final_menu_path}")